In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import HilbertPhaseShiftRegister
from PyHEADTAIL.feedback.processors.signal import BeamParameters

np.random.seed(0)

In [ ]:
""" 
    In this example, a feedback system consisting of multiple pickups and kickers is demonstrated. It differs
    very little from the previous example (002_separated_pickup_and_kicker), but it might look messy
    because multiple pickups and kickers must be initialized.   
"""

In [ ]:
# Basic parameters for the simulation
n_macroparticles = 10000
n_slices = 20
n_segments = 5
n_sigma_z = 3 

n_turns = 400

# Longitudinal motion of the bunch is not taken into account in this example.
machine = Machine(Q_s = 0.00000001)
# The longitudinal motion can be included to simulations by uncommenting the following line
# machine = Machine(Q_s = 0.0020443)

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles,n_segments, 
                                                             n_slices,n_sigma_z)
bunch_unkicked = copy.deepcopy(bunch_ref)

In [ ]:
# This creates an artificially kicked bunch, which will be damped with different feedback systems

slice_set = bunch_ref.get_slices(slicer_ref, statistics=True)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

# random kicks
kick_x = 0.003*(-1.0+2*np.random.rand(n_slices))
kick_y = 0.003*(-1.0+2*np.random.rand(n_slices))

for p_id, s_id in itertools.izip(p_idx,s_idx):
    bunch_ref.x[p_id] += kick_x[s_id]
    bunch_ref.y[p_id] += kick_y[s_id]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map] + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:
feedback_gain = 0.01
# feedback_gain = (0.01,0.01)

# Parameters for the registers
delay = 1 
n_values = 3

In [ ]:
# This creates an artificially kicked bunch, which will be damped by using different feedback systems

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'average'),
    Averager()
]

# Because two kickers are used in this example, two times higher gain is used in this reference system
# which consists of one kicker
feedback_map = OneboxFeedback(2.*feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)
# If you don't understand details of the code above, please study examples from 
# the file 001_ideal_feedbacks.ipynb

In [ ]:
bunch_multi_object = copy.deepcopy(bunch_ref)
tracker_multi_object = BunchTracker(bunch_multi_object)
slicer_multi_object = copy.deepcopy(slicer_ref)

In [ ]:
# Three pickups are initialized in this cell. The exactly same code as in the previous example
# (002_separated_pickup_and_kicker.ipynb) is also used in this example

# In this example, it is chosen that the pickups are located after the first, second and third segments of the trans_map.
# Trans_map is divded into a number of segments determined in the variable 'n_segments'

##############
## Pickup 1 ##
##############
processors_pickup_1_x = [
    ChargeWeighter(normalization = 'average'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_pickup_1_y = [
    ChargeWeighter(normalization = 'average'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]
pickup_1_beam_parameters_x = BeamParameters(1.*2.*pi/float(n_segments)*machine.Q_x, machine.beta_x_inj)
pickup_1_beam_parameters_y = BeamParameters(1.*2.*pi/float(n_segments)*machine.Q_y, machine.beta_y_inj)

pickup_1_map = PickUp(slicer_multi_object,
                    processors_pickup_1_x,processors_pickup_1_y, 
                    pickup_1_beam_parameters_x, pickup_1_beam_parameters_y)


##############
## Pickup 2 ##
##############
processors_pickup_2_x = [
    ChargeWeighter(normalization = 'average'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_pickup_2_y = [
    ChargeWeighter(normalization = 'average'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

pickup_2_beam_parameters_x = BeamParameters(2.*2.*pi/float(n_segments)*machine.Q_x, machine.beta_x_inj)
pickup_2_beam_parameters_y = BeamParameters(2.*2.*pi/float(n_segments)*machine.Q_y, machine.beta_y_inj)

pickup_2_map = PickUp(slicer_multi_object,
                    processors_pickup_2_x,processors_pickup_2_y, 
                    pickup_2_beam_parameters_x, pickup_2_beam_parameters_y)


##############
## Pickup 3 ##
##############
processors_pickup_3_x = [
    ChargeWeighter(normalization = 'average'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_pickup_3_y = [
    ChargeWeighter(normalization = 'average'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

pickup_3_beam_parameters_x = BeamParameters(3.*2.*pi/float(n_segments)*machine.Q_x, machine.beta_x_inj)
pickup_3_beam_parameters_y = BeamParameters(3.*2.*pi/float(n_segments)*machine.Q_y, machine.beta_y_inj)

pickup_3_map = PickUp(slicer_multi_object,
                    processors_pickup_3_x,processors_pickup_3_y, 
                    pickup_3_beam_parameters_x, pickup_3_beam_parameters_y)

In [ ]:
# Two kickers are initialized in this cell. The exactly same code as in the previous example
# (002_separated_pickup_and_kicker.ipynb) is also used in this example. The only difference is that references for
# multiple register are given as an input parameter.
#
# In this example, it is chosen that the pickups are located after the first, second and fourth segment of the trans_map.
# Furthermore, it was chosen that Kicker 1 utilizes signals from pickups 1 and 2 and Kicker 2 utilizes signals from 
# the pickups 1 and 3.

##############
## Kicker 1 ##
##############
processors_kicker_1_x = [Bypass()]
processors_kicker_1_y = [Bypass()]

registers_kicker_1_x = [
    processors_pickup_1_x[2],
    processors_pickup_2_x[2]
]
registers_kicker_1_y = [
    processors_pickup_1_y[2],
    processors_pickup_2_y[2]
]

kicker_1_beam_parameters_x = BeamParameters(2.*2.*pi/float(n_segments)*machine.Q_x, machine.beta_x_inj)
kicker_1_beam_parameters_y = BeamParameters(2.*2.*pi/float(n_segments)*machine.Q_y, machine.beta_y_inj)

kicker_1_map = Kicker(feedback_gain, slicer_multi_object,
                    processors_kicker_1_x, processors_kicker_1_y,
                    registers_kicker_1_x, registers_kicker_1_y, 
                    kicker_1_beam_parameters_x, kicker_1_beam_parameters_y)

##############
## Kicker 2 ##
##############
processors_kicker_2_x = [Bypass()]
processors_kicker_2_y = [Bypass()]

registers_kicker_2_x = [
    processors_pickup_1_x[2],
    processors_pickup_3_x[2]
]
registers_kicker_2_y = [
    processors_pickup_1_y[2],
    processors_pickup_3_y[2]
]

kicker_2_beam_parameters_x = BeamParameters(4.*2.*pi/float(n_segments)*machine.Q_x, machine.beta_x_inj)
kicker_2_beam_parameters_y = BeamParameters(4.*2.*pi/float(n_segments)*machine.Q_y, machine.beta_y_inj)

kicker_2_map = Kicker(feedback_gain, slicer_multi_object,
                    processors_kicker_2_x, processors_kicker_2_y,
                    registers_kicker_2_x, registers_kicker_2_y, 
                    kicker_2_beam_parameters_x, kicker_2_beam_parameters_y)

In [ ]:
# After that the maps of the pickups and the kickers must be added to correct slots of the total map 
# determined by the locations of the picups and the kickers
total_map_multi_object = [trans_map[0]] + [pickup_1_map] + [trans_map[1]] + [pickup_2_map] + [kicker_1_map] + [trans_map[2]]+ [pickup_3_map] + [trans_map[3]] + [kicker_2_map]
for element in trans_map[4:]:
    total_map_multi_object += [element]
total_map_multi_object += [long_map]
    
track(n_turns, bunch_multi_object,total_map_multi_object,tracker_multi_object)

In [ ]:
# In here, the traces and the projections from different implementations of the feedback system are compared.
# Note the scale in the emittance figures.

compare_traces([tracker_OneBox_bunch,tracker_multi_object],
               ['Ideal', 'Multiple pickups and kickers'])
compare_projections([ bunch_OneBox_bunch,  bunch_multi_object], 
                    ['Ideal', 'Multiple pickups and kickers'])

Jani Komppula, CERN, 2017